# Timing Analysis 

In [23]:
from timeit import timeit 
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.neighbors import NearestNeighbors
from scipy import sparse
import scipy
import submodlib_cpp as subcp
from submodlib.helper import create_kernel
import random
from time import time

## 1) CPP Helper V/S Python Helper

In [24]:
data = None
try: #Simply load data if already created
    with open('/content/drive/MyDrive/submodlib_data/large_data.npy', 'rb') as f:
        data = np.load(f, allow_pickle=True)
except: #Create data
    random.seed(0)
    def generate_datapoint(arg):
        #random.seed(arg)
        #x=random.randrange(1000)-random.randrange(1000)
        #y=random.randrange(10)-random.randrange(10)
        #z=(random.randrange(100)/100)-(random.randrange(100)/100)
        l_feat = []
        for i in range(2000):
            toss = random.randrange(0, 2)
            if toss==0:
                bound_val=random.randrange(10, 10000)
                #val=random.randrange(bound_val)-random.randrange(bound_val)
                val=random.randrange(-1*bound_val, bound_val)
            else:
                val = random.randrange(-100, 100)/100

            l_feat.append(val)
        if arg%10000==0:
            print(arg)
        #return (x,y,z)
        return l_feat

    l = list(map(generate_datapoint, range(100000)))#Generating 100K datapoints
    data = np.array(l)
    
    with open('/content/drive/MyDrive/submodlib_data/large_data.npy', 'wb') as f:
        np.save(f,data, allow_pickle=True)

l_record=[]

In [25]:
np.shape(data)

(100000, 2000)

In [26]:
def fun1():# cpp_helper_euclidean (Non-vectorized, min-heap based approach)
    subcp.create_kernel(data.tolist(), "euclidean" ,num_neigh)
t=timeit('fun1', 'from __main__ import fun1')
l_record.append(("cpp_helper_euclidean", t))
print("cpp_helper_euclidean:", t,'\n')

cpp_helper_euclidean: 0.011599696000303084 



In [27]:
def fun2(): #python_helper_euclidean(vectorized knn clustering approach) 
    n_, K_dense = create_kernel(data, 'dense','euclidean')
t=timeit('fun2', 'from __main__ import fun2')
l_record.append(("python_helper_euclidean", t))
print("python_helper_euclidean:", t,'\n')

python_helper_euclidean: 0.01354807500001698 



In [28]:
def fun3():# cpp_helper_cosine (Non-vectorized, min-heap based approach)
    subcp.create_kernel(data.tolist(), "cosine" ,num_neigh)
t=timeit('fun3', 'from __main__ import fun3')
l_record.append(("cpp_helper_cosine", t))
print("cpp_helper_cosine:", t,'\n')

cpp_helper_cosine: 0.019444560000010824 



In [29]:
def fun4(): #python_helper_cosine(vectorized knn clustering approach) 
    n_, K_dense = create_kernel(data, 'dense','cosine')
t=timeit('fun4', 'from __main__ import fun4')
l_record.append(("python_helper_cosine", t))
print("python_helper_cosine:", t,'\n')

python_helper_cosine: 0.011354927999946085 



**CPP Helper V/S Python Helper: SUMMARY**

In [30]:
df = pd.DataFrame(columns = ['name', 'time'],data=l_record)
df

,name,time
0,cpp_helper_euclidean,0.011600
1,python_helper_euclidean,0.013548
2,cpp_helper_cosine,0.019445
3,python_helper_cosine,0.011355
